### Feature Engineering y Modelado Predictivo de Permisos de Circulación

Este notebook toma el dataset limpio y estandarizado del paso anterior y lo utiliza para un problema de Machine Learning. El objetivo es:

1. Ingeniería de Características: Crear nuevas variables que puedan mejorar el rendimiento del modelo.

2. Preparación de Datos: Preparar los datos para el entrenamiento de un modelo predictivo.

3. Modelado: Entrenar un modelo de clasificación simple para predecir el estado de un permiso (Activo o Inactivo) basándose en sus características.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar el dataset limpio y procesado del paso anterior
df_clean = pd.read_csv('data/processed/permisos_circulacion_limpio.csv')

# Inspección inicial del dataset limpio
print("--- Información del dataset limpio ---")
df_clean.info()
print("\n--- Primeras 5 filas del dataset limpio ---")
print(df_clean.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/permisos_circulacion_limpio.csv'

### 1. Ingeniería de Características (Feature Engineering)
Vamos a crear nuevas variables a partir de las existentes para darle más información al modelo. Transformaremos las fechas en variables de tiempo y usaremos 'one-hot encoding' para las variables categóricas.

In [ ]:
# Convertir la columna de fecha a formato datetime
df_clean['fecha_emision'] = pd.to_datetime(df_clean['fecha_emision'], errors='coerce')

# Eliminar las filas con fechas inválidas que no pudieron ser convertidas
df_clean.dropna(subset=['fecha_emision'], inplace=True)

# Crear nuevas variables de tiempo
df_clean['dia_semana'] = df_clean['fecha_emision'].dt.day_name()
df_clean['mes'] = df_clean['fecha_emision'].dt.month
df_clean['dia_del_mes'] = df_clean['fecha_emision'].dt.day

# Calcular una fecha de expiración para un posible problema de clasificación
df_clean['fecha_expiracion'] = df_clean['fecha_emision'] + pd.to_timedelta(df_clean['duracion_dias'], unit='D')

# Convertir variables categóricas en variables 'dummy' (one-hot encoding)
df_model = pd.get_dummies(df_clean, columns=['tipo_vehiculo', 'zona_circulacion', 'dia_semana'])

# Eliminar la columna de fecha_emision ya que hemos extraído la información relevante
df_model.drop(columns=['fecha_emision', 'fecha_expiracion'], inplace=True)

# Verificación de la estructura del nuevo DataFrame
print("--- Estructura del DataFrame después de la ingeniería de características ---")
print(df_model.head())
print("\n--- Columnas finales para el modelo ---")
print(df_model.columns)

### 2. Preparación de los Datos para el Modelado

Dividiremos nuestro dataset en dos partes: un conjunto de entrenamiento (train) y un conjunto de prueba (test). Esto es fundamental para evaluar el rendimiento del modelo de forma objetiva.

1. Características (X): Las variables que usaremos para predecir.

2. Variable Objetivo (y): La variable que queremos predecir, en este caso, el estado del permiso.

In [ ]:
# Definir las características (X) y la variable objetivo (y)
X = df_model.drop(columns=['estado'])
y = df_model['estado']

# Convertir la variable objetivo a valores numéricos (0 y 1)
y = y.map({'Activo': 1, 'Inactivo': 0})

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("--- Tamaño de los conjuntos de datos ---")
print(f"Conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Conjunto de prueba (X_test): {X_test.shape}")

### 3. Entrenamiento y Evaluación del Modelo

Utilizaremos un modelo de clasificación simple y robusto como RandomForestClassifier. Luego evaluaremos su rendimiento en el conjunto de prueba para ver qué tan bien predice el estado del permiso.

In [ ]:
# Inicializar y entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Inactivo', 'Activo'])
conf_matrix = confusion_matrix(y_test, y_pred)

print("--- Evaluación del modelo RandomForest ---")
print(f"Precisión (Accuracy): {accuracy:.4f}")
print("\n--- Reporte de Clasificación ---")
print(report)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Inactivo', 'Activo'], yticklabels=['Inactivo', 'Activo'])
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión')
plt.show()

### Conclusión y Próximos Pasos

Este ejercicio demuestra cómo un proceso de feature engineering y modelado puede extraer valor de datos limpios. El modelo de Random Forest logra una buena precisión, lo que indica que las características que creamos son útiles para predecir el estado de un permiso.

Para mejorar este proyecto en el futuro, se podría:

* Realizar una optimización de hiperparámetros del modelo.

* Probar otros modelos de clasificación (como Gradient Boosting).

* Realizar un análisis de la importancia de las características para entender qué variables tienen un mayor impacto en la predicción.

* Visualizar los resultados de las predicciones en un mapa (si tuviéramos coordenadas geográficas más detalladas).